In [7]:
from pathlib import Path
import pandas as pd

In [8]:
csv_file_dir = Path("../../content/tables/TRP/")
params_to_keep = set(["CRRA", "BeqFac", "BeqShift", "WealthShare", "criterion"])

# Get all files in the directory
files = [f for f in csv_file_dir.iterdir() if f.is_file()]

parameters = []

# Iterate over each file
for file in files:
    file_name = file.stem.replace("_estimate_results", "")
    # Read the CSV file and convert it to a dictionary
    res = pd.read_csv(file, header=None)
    res = res.set_index(res.columns[0])[res.columns[1]].to_dict()

    # Create a new dictionary for this file
    file_parameters = {}

    labor = 0
    stock = 0

    if "Sub" in file_name:
        if "(Labor)" in file_name:
            labor = 1
            file_name = file_name.replace("(Labor)", "")

        if "(Stock)" in file_name:
            stock = 1
            file_name = file_name.replace("(Stock)", "")

    file_name = file_name.replace("Sub", "").replace("Market", "")
    file_parameters["Name"] = file_name
    file_parameters["Stock"] = stock
    file_parameters["Labor"] = labor

    # Iterate over each parameter we want to keep
    for param in params_to_keep:
        # If the parameter is in the dictionary, add it to the file's parameters
        if param in res:
            file_parameters[param] = res[param]

    # Add the file's parameters to the overall parameters dictionary
    parameters.append(file_parameters)

In [9]:
def format_df(df):
    for col in df.columns:
        # Check if column is of float type
        if col in params_to_keep:
            df[col] = df[col].astype(float).round(3).fillna("")
            pass
        # Check if column contains only 0 and 1
        else:
            df[col] = df[col].map({0: "", 1: "✔️"})
    return df


df = pd.DataFrame(parameters).set_index("Name")
format_df(df)

,Stock,Labor,CRRA,criterion,BeqFac,BeqShift,WealthShare
Name,,,,,,,
IndShock,,✔️,20.000,544.528,,,
IndShock,✔️,✔️,20.000,557.437,,,
IndShock,✔️,,5.127,534.345,,,
IndShock,,,4.476,509.235,,,
Portfolio,,✔️,18.056,1.458,,,
Portfolio,✔️,✔️,6.091,1.506,,,
Portfolio,✔️,,2.855,3.072,,,
Portfolio,,,15.561,2.697,,,
WarmGlowPortfolio,,✔️,13.826,0.872,63.419,61.308,
